### Задача 48 "Ортогональные латинские квадраты"
Сформулируйте задачу построения ортогональных латинских квадратов как задачу целочисленного(булева) линейного программирования. Неизвестными задачи будут булевы переменные Xijkl, где индексы i, j, k, l меняются от 1 до n, причем Xijkl <=> в позиции (i,j) стоит пара (k,l).
Задача имеет следующие ограничения :
Запишите условия задачи, используя язык моделирования, например, из библиотеки pulp или другой подобной. Решите задачу средствами библиотеки для n = 2, 3, 4, 5, 6,10. Сколько времени тратит ваша программа?


Установим модуль pulp  и подключим datetime для рассчёта времени: 

In [ ]:
!pip install pulp
from datetime import datetime
from pulp import *

Создадим модель и переменные, определим целевую функцию:

In [ ]:
for n in range(2, 11):
    # Инициализируем модель
    model = pulp.LpProblem(name="orthogonal_latin_squares", sense=LpMinimize)

    # Создаем переменные
    X = pulp.LpVariable.dicts("X", [(i, j, k, l) for i in range(1, n + 1)
                                         for j in range(1, n + 1)
                                         for k in range(1, n + 1)
                                         for l in range(1, n + 1)], cat="Binary")


    # Целевая функция
    model += 0, "Dummy Objective"

Далее введём все необходимые ограничения:

In [ ]:
 # Ограничения для каждой клетки
    for i in range(1, n+1):
        for j in range(1, n+1):
            model += pulp.lpSum(X[i,j,k,l] for k in range(1, n+1) for l in range(1, n+1)) == 1, f"Cell ({i}, {j})"

    # Ограничения на элементы в строках
    for i in range(1, n+1):
        for k in range(1, n+1):
            model += pulp.lpSum(X[i,j,k,l] for j in range(1, n+1) for l in range(1, n+1)) == 1, f"Row {i}, Symbol {k}"

    # Ограничения на элементы в столбцах
    for j in range(1, n+1):
        for l in range(1, n+1):
            model += pulp.lpSum(X[i,j,k,l] for i in range(1, n+1) for k in range(1, n+1)) == 1, f"Column {j}, Symbol {l}"        
            
    # Ограничения на ортогональность квадратов
    for k in range(1, n+1):
        for l in range(1, n+1):
            for i in range(1, n+1):
                model += pulp.lpSum(X[i,j,k,l] for j in range(1, n+1)) == pulp.lpSum(X[i,j,l,k] for j in range(1, n+1)), f"Orthogonality {k}, {l}, Row {i}"
            for j in range(1, n+1):
                model += pulp.lpSum(X[i,j,k,l] for i in range(1, n+1)) == pulp.lpSum(X[i,j,l,k] for i in range(1, n+1)), f"Orthogonality {k}, {l}, Column {j}"
  

Теперь можно приступить к решению конкретной задачи и подсчёту затраченного времени

In [ ]:
# Решаем задачу
    start_time = datetime.now()
    model.solve()
    end_time = datetime.now()

Результат выводим с помощью циклов:

In [3]:
    # Выводим результаты
    print(f"Решение для n = {n}")
    if model.status == LpStatusOptimal and n != 2 and n != 6 :
        print("Оптимальное решение найдено")
        print(f"Время выполнения: {end_time - start_time} сек")
        print("Первый квадрат:")
        c = 0
        for i in range(1, n + 1):
            for j in range(1, n + 1):
                for k in range(1, n + 1):
                    for l in range(1, n + 1):
                        if X[(i, j, k, l)].value() == 1:
                            if c < n:
                                print(f"({k}, {l})", end=" ")
                                c += 1
                            else:
                                print("\n")
                                print(f"({k}, {l})", end=" ")
                                c = 1
                            
        print("\n")     
        print("Второй квадрат:")
        for i in range(1, n + 1):
            for j in range(1, n + 1):
                for k in range(1, n + 1):
                    for l in range(1, n + 1):
                        if X[(i, j, k, l)].value() == 1:
                            if c < n:
                                print(f"({l}, {k})", end=" ")
                                c += 1
                            else:
                                print("\n")
                                print(f"({l}, {k})", end=" ")
                                c = 1
        print("\n")
    else:
        print("Решение не найдено")
    print("------------------------------")

Решение для n = 10
Оптимальное решение найдено
Время выполнения: 0:02:08.858166 сек
Первый квадрат:
(1, 1) (10, 5) (2, 2) (8, 6) (5, 10) (7, 7) (4, 4) (3, 3) (9, 9) (6, 8) 

(5, 5) (7, 1) (4, 10) (10, 4) (9, 9) (3, 3) (6, 8) (1, 7) (2, 2) (8, 6) 

(2, 2) (5, 10) (3, 3) (6, 8) (1, 4) (4, 1) (7, 7) (10, 5) (8, 6) (9, 9) 

(9, 9) (8, 8) (6, 6) (1, 1) (2, 3) (10, 5) (3, 2) (5, 10) (4, 4) (7, 7) 

(6, 7) (3, 3) (8, 8) (2, 2) (7, 6) (1, 4) (10, 10) (9, 9) (5, 5) (4, 1) 

(8, 8) (1, 7) (10, 4) (4, 10) (3, 2) (9, 9) (2, 3) (6, 6) (7, 1) (5, 5) 

(10, 3) (6, 6) (5, 9) (7, 7) (4, 1) (8, 2) (9, 5) (2, 8) (3, 10) (1, 4) 

(3, 10) (2, 2) (7, 1) (9, 5) (8, 8) (6, 6) (5, 9) (4, 4) (1, 7) (10, 3) 

(4, 4) (9, 9) (1, 7) (3, 3) (10, 5) (5, 10) (8, 6) (7, 1) (6, 8) (2, 2) 

(7, 6) (4, 4) (9, 5) (5, 9) (6, 7) (2, 8) (1, 1) (8, 2) (10, 3) (3, 10) 

Второй квадрат:


(1, 1) (5, 10) (2, 2) (6, 8) (10, 5) (7, 7) (4, 4) (3, 3) (9, 9) (8, 6) 

(5, 5) (1, 7) (10, 4) (4, 10) (9, 9) (3, 3) (8, 6) (7, 1) (2, 2) (6,

### Вывод: 
как мы видим, при увеличении n время выполнения программы возрастает в связи с увеличением как количества переменных, так и количества ограничений.
Например, для n = 10 время выполнения составляет 0:02:46.602860 сек,  количество переменных равно 10^4, а ограничений - 10^6.
Поэтому на выполнение программы для больших значений n требуется очень много времени.


Решение для n = 2
Решение не найдено

Решение для n = 3
Оптимальное решение найдено
Время выполнения: 0:00:00.031247 сек
Первый квадрат:
(2, 1) (1, 2) (3, 3) 

(1, 2) (3, 3) (2, 1) 

(3, 3) (2, 1) (1, 2) 

Второй квадрат:


(1, 2) (2, 1) (3, 3) 

(2, 1) (3, 3) (1, 2) 

(3, 3) (1, 2) (2, 1) 

------------------------------
Решение для n = 4
Оптимальное решение найдено
Время выполнения: 0:00:00.051112 сек
Первый квадрат:
(1, 1) (4, 4) (3, 3) (2, 2) 

(2, 2) (3, 3) (1, 1) (4, 4) 

(3, 3) (2, 2) (4, 4) (1, 1) 

(4, 4) (1, 1) (2, 2) (3, 3) 

Второй квадрат:


(1, 1) (4, 4) (3, 3) (2, 2) 

(2, 2) (3, 3) (1, 1) (4, 4) 

(3, 3) (2, 2) (4, 4) (1, 1) 

(4, 4) (1, 1) (2, 2) (3, 3) 

------------------------------
Решение для n = 5
Оптимальное решение найдено
Время выполнения: 0:00:00.068313 сек
Первый квадрат:
(5, 5) (2, 4) (3, 3) (1, 1) (4, 2) 

(1, 1) (4, 2) (5, 5) (2, 4) (3, 3) 

(3, 3) (1, 1) (2, 4) (4, 2) (5, 5) 

(2, 4) (3, 5) (4, 2) (5, 3) (1, 1) 

(4, 2) (5, 3) (1, 1) (3, 5) (2, 4) 

Второй квадрат:


(5, 5) (4, 2) (3, 3) (1, 1) (2, 4) 

(1, 1) (2, 4) (5, 5) (4, 2) (3, 3) 

(3, 3) (1, 1) (4, 2) (2, 4) (5, 5) 

(4, 2) (5, 3) (2, 4) (3, 5) (1, 1) 

(2, 4) (3, 5) (1, 1) (5, 3) (4, 2) 

------------------------------
Решение для n = 6
Решение не найдено

Решение для n = 7
Оптимальное решение найдено
Время выполнения: 0:00:00.678731 сек
Первый квадрат:
(4, 4) (3, 7) (7, 3) (1, 1) (5, 5) (2, 2) (6, 6) 

(3, 3) (5, 5) (1, 2) (6, 7) (2, 1) (7, 6) (4, 4) 

(7, 7) (2, 1) (6, 6) (5, 5) (3, 3) (4, 4) (1, 2) 

(1, 1) (6, 6) (5, 5) (2, 2) (4, 4) (3, 3) (7, 7) 

(2, 2) (7, 3) (3, 7) (4, 4) (6, 6) (1, 1) (5, 5) 

(5, 5) (4, 4) (2, 1) (7, 6) (1, 2) (6, 7) (3, 3) 

(6, 6) (1, 2) (4, 4) (3, 3) (7, 7) (5, 5) (2, 1) 

Второй квадрат:


(4, 4) (7, 3) (3, 7) (1, 1) (5, 5) (2, 2) (6, 6) 

(3, 3) (5, 5) (2, 1) (7, 6) (1, 2) (6, 7) (4, 4) 

(7, 7) (1, 2) (6, 6) (5, 5) (3, 3) (4, 4) (2, 1) 

(1, 1) (6, 6) (5, 5) (2, 2) (4, 4) (3, 3) (7, 7) 

(2, 2) (3, 7) (7, 3) (4, 4) (6, 6) (1, 1) (5, 5) 

(5, 5) (4, 4) (1, 2) (6, 7) (2, 1) (7, 6) (3, 3) 

(6, 6) (2, 1) (4, 4) (3, 3) (7, 7) (5, 5) (1, 2) 

------------------------------
Решение для n = 8
Оптимальное решение найдено
Время выполнения: 0:00:05.625333 сек
Первый квадрат:
(1, 3) (8, 8) (4, 4) (3, 1) (5, 5) (7, 7) (2, 6) (6, 2) 

(5, 4) (6, 2) (2, 6) (4, 5) (7, 7) (3, 3) (8, 8) (1, 1) 

(8, 8) (7, 7) (6, 2) (5, 4) (1, 3) (4, 5) (3, 1) (2, 6) 

(3, 1) (5, 5) (7, 7) (2, 2) (8, 4) (6, 6) (1, 3) (4, 8) 

(4, 5) (2, 6) (8, 8) (1, 3) (3, 1) (5, 4) (6, 2) (7, 7) 

(2, 2) (4, 4) (5, 3) (8, 8) (6, 6) (1, 1) (7, 7) (3, 5) 

(6, 6) (3, 3) (1, 1) (7, 7) (4, 8) (2, 2) (5, 5) (8, 4) 

(7, 7) (1, 1) (3, 5) (6, 6) (2, 2) (8, 8) (4, 4) (5, 3) 

Второй квадрат:


(3, 1) (8, 8) (4, 4) (1, 3) (5, 5) (7, 7) (6, 2) (2, 6) 

(4, 5) (2, 6) (6, 2) (5, 4) (7, 7) (3, 3) (8, 8) (1, 1) 

(8, 8) (7, 7) (2, 6) (4, 5) (3, 1) (5, 4) (1, 3) (6, 2) 

(1, 3) (5, 5) (7, 7) (2, 2) (4, 8) (6, 6) (3, 1) (8, 4) 

(5, 4) (6, 2) (8, 8) (3, 1) (1, 3) (4, 5) (2, 6) (7, 7) 

(2, 2) (4, 4) (3, 5) (8, 8) (6, 6) (1, 1) (7, 7) (5, 3) 

(6, 6) (3, 3) (1, 1) (7, 7) (8, 4) (2, 2) (5, 5) (4, 8) 

(7, 7) (1, 1) (5, 3) (6, 6) (2, 2) (8, 8) (4, 4) (3, 5) 

------------------------------
Решение для n = 9
Оптимальное решение найдено
Время выполнения: 0:00:03.100354 сек
Первый квадрат:
(4, 4) (1, 1) (6, 6) (2, 2) (9, 9) (7, 7) (5, 5) (8, 8) (3, 3) 

(2, 9) (9, 2) (8, 8) (7, 3) (5, 5) (6, 6) (1, 1) (3, 7) (4, 4) 

(9, 2) (2, 9) (1, 1) (5, 5) (8, 7) (3, 3) (4, 4) (6, 6) (7, 8) 

(5, 5) (7, 7) (2, 2) (8, 8) (3, 3) (1, 1) (6, 6) (4, 4) (9, 9) 

(8, 7) (6, 6) (9, 9) (1, 1) (7, 8) (4, 4) (3, 3) (2, 2) (5, 5) 

(7, 8) (4, 4) (3, 3) (6, 6) (1, 1) (5, 5) (8, 7) (9, 9) (2, 2) 

(3, 3) (5, 5) (4, 4) (9, 9) (6, 6) (2, 2) (7, 8) (1, 1) (8, 7) 

(6, 1) (3, 3) (7, 7) (4, 4) (2, 2) (8, 8) (9, 9) (5, 5) (1, 6) 

(1, 6) (8, 8) (5, 5) (3, 7) (4, 4) (9, 9) (2, 2) (7, 3) (6, 1) 

Второй квадрат:


(4, 4) (1, 1) (6, 6) (2, 2) (9, 9) (7, 7) (5, 5) (8, 8) (3, 3) 

(9, 2) (2, 9) (8, 8) (3, 7) (5, 5) (6, 6) (1, 1) (7, 3) (4, 4) 

(2, 9) (9, 2) (1, 1) (5, 5) (7, 8) (3, 3) (4, 4) (6, 6) (8, 7) 

(5, 5) (7, 7) (2, 2) (8, 8) (3, 3) (1, 1) (6, 6) (4, 4) (9, 9) 

(7, 8) (6, 6) (9, 9) (1, 1) (8, 7) (4, 4) (3, 3) (2, 2) (5, 5) 

(8, 7) (4, 4) (3, 3) (6, 6) (1, 1) (5, 5) (7, 8) (9, 9) (2, 2) 

(3, 3) (5, 5) (4, 4) (9, 9) (6, 6) (2, 2) (8, 7) (1, 1) (7, 8) 

(1, 6) (3, 3) (7, 7) (4, 4) (2, 2) (8, 8) (9, 9) (5, 5) (6, 1) 

(6, 1) (8, 8) (5, 5) (7, 3) (4, 4) (9, 9) (2, 2) (3, 7) (1, 6) 

------------------------------
Решение для n = 10
Оптимальное решение найдено
Время выполнения: 0:03:12.560132 сек
Первый квадрат:
(1, 1) (10, 5) (2, 2) (8, 6) (5, 10) (7, 7) (4, 4) (3, 3) (9, 9) (6, 8) 

(5, 5) (7, 1) (4, 10) (10, 4) (9, 9) (3, 3) (6, 8) (1, 7) (2, 2) (8, 6) 

(2, 2) (5, 10) (3, 3) (6, 8) (1, 4) (4, 1) (7, 7) (10, 5) (8, 6) (9, 9) 

(9, 9) (8, 8) (6, 6) (1, 1) (2, 3) (10, 5) (3, 2) (5, 10) (4, 4) (7, 7) 

(6, 7) (3, 3) (8, 8) (2, 2) (7, 6) (1, 4) (10, 10) (9, 9) (5, 5) (4, 1) 

(8, 8) (1, 7) (10, 4) (4, 10) (3, 2) (9, 9) (2, 3) (6, 6) (7, 1) (5, 5) 

(10, 3) (6, 6) (5, 9) (7, 7) (4, 1) (8, 2) (9, 5) (2, 8) (3, 10) (1, 4) 

(3, 10) (2, 2) (7, 1) (9, 5) (8, 8) (6, 6) (5, 9) (4, 4) (1, 7) (10, 3) 

(4, 4) (9, 9) (1, 7) (3, 3) (10, 5) (5, 10) (8, 6) (7, 1) (6, 8) (2, 2) 

(7, 6) (4, 4) (9, 5) (5, 9) (6, 7) (2, 8) (1, 1) (8, 2) (10, 3) (3, 10) 

Второй квадрат:


(1, 1) (5, 10) (2, 2) (6, 8) (10, 5) (7, 7) (4, 4) (3, 3) (9, 9) (8, 6) 

(5, 5) (1, 7) (10, 4) (4, 10) (9, 9) (3, 3) (8, 6) (7, 1) (2, 2) (6, 8) 

(2, 2) (10, 5) (3, 3) (8, 6) (4, 1) (1, 4) (7, 7) (5, 10) (6, 8) (9, 9) 

(9, 9) (8, 8) (6, 6) (1, 1) (3, 2) (5, 10) (2, 3) (10, 5) (4, 4) (7, 7) 

(7, 6) (3, 3) (8, 8) (2, 2) (6, 7) (4, 1) (10, 10) (9, 9) (5, 5) (1, 4) 

(8, 8) (7, 1) (4, 10) (10, 4) (2, 3) (9, 9) (3, 2) (6, 6) (1, 7) (5, 5) 

(3, 10) (6, 6) (9, 5) (7, 7) (1, 4) (2, 8) (5, 9) (8, 2) (10, 3) (4, 1) 

(10, 3) (2, 2) (1, 7) (5, 9) (8, 8) (6, 6) (9, 5) (4, 4) (7, 1) (3, 10) 

(4, 4) (9, 9) (7, 1) (3, 3) (5, 10) (10, 5) (6, 8) (1, 7) (8, 6) (2, 2) 

(6, 7) (4, 4) (5, 9) (9, 5) (7, 6) (8, 2) (1, 1) (2, 8) (3, 10) (10, 3) 

------------------------------